This code is for re-creating the selected subset of images after piloting and finding images that the mean accuracy rate is bellow 0.5 

- find the selected images, layer and mapped number
- delete the selected images from the contrast similarity df  
- re-select the most similar images 

In [24]:
from copyreg import pickle
import pandas as pd 
import numpy as np 
import pickle
import pathlib as path
import os 

In [25]:
bad_pictures = ['2874299429-139694.jpg','2883273443-68193.jpg','2986451363-73598.jpg','2964222378-71332.jpg']

In [26]:
similarity_df=pd.read_csv('similarity_fungi_all.csv')
similarity_df.drop(columns=['Unnamed: 0','model','category1','category2'],inplace=True)

In [45]:
with open('contrast_similarity_df.pickle', 'rb') as handle:
    return_dict = pickle.load(handle)

contrast_similarity_per_layer=return_dict['contrast_similarity_per_layer']
mapping_dict=return_dict['mapping_dict']
# changing key names to end with .jpg and not .pkl
ini_list = mapping_dict.keys()
ini_list = [x.replace('.pkl','.jpg') for x in ini_list ]
mapping_dict = dict(zip(ini_list, list(mapping_dict.values())))

contrast_per_layer=return_dict['contrast_per_layer']
all_layers_matrices=return_dict['all_layers_matrices']

In [46]:
# creating a dictionary with the keys as the target images names and the values are the layer
PATH_TARGET_IMAGES_FOLDER = path.Path(r'..\evaluation\selected')
selected_images_paths = list(PATH_TARGET_IMAGES_FOLDER.glob("*/*.jpg"))

target_image_layer_dic = {}
for x in selected_images_paths:
    layer_number = x.parent.name.split("_")[-1]
    image_name = x.name
    # 
    target_image_layer_dic[image_name] = layer_number


In [47]:
# creating df with the layer and mapped value of each target image
target_image_layer_df = pd.DataFrame.from_dict(target_image_layer_dic, orient='index',
                       columns=['layer'])
total = []
for key in target_image_layer_dic.keys():
    mapped_value = mapping_dict[key]
    total.append(mapped_value)


target_image_layer_df['mapped_value'] = total

1) creating new target_image_layer_df without bad images , so we know what new images we need to find 
2) creating contrast_similarity_updated without used images so we cant choose them again 

In [65]:
mapping_dict['2964224310-219528.jpg']

319

In [48]:
# we need to make sure we dont choose distractors that have alraedy been chosen for target images and the target images themselves :
contrast_similarity_updated = contrast_similarity_per_layer.copy()

PATH_ALL_PILOT_IMAGES_FOLDER = path.Path(r'..\task\pilot_images')
old_images = list(PATH_ALL_PILOT_IMAGES_FOLDER.glob("*.jpg"))
old_images = [x.name for x in old_images]

total_taget_distarctors_mapped = []
for x in old_images:
   
    total_taget_distarctors_mapped.append(mapping_dict[x])

for image_val in total_taget_distarctors_mapped:
    contrast_similarity_updated[image_val,:,:] = -9999

In [68]:
contrast_similarity_updated[:,319,0]

array([ 1.07178934e+00, -9.99900000e+03, -1.47774407e-01, ...,
        8.23993638e-01,  1.06076563e+00, -1.08953010e-01])

In [50]:
# updating new df without bad pictures 
# moving bad pictures to new folder 
target_image_layer_df_updated = target_image_layer_df.copy()
for picture in bad_pictures:

    id_bad = target_image_layer_df_updated[target_image_layer_df_updated.index == picture]
    target_image_layer_df_updated = target_image_layer_df_updated.drop(index=id_bad.index)

    # moving image to new folder - unselected 
    
    old_path = PATH_TARGET_IMAGES_FOLDER / path.Path('layer_' + id_bad['layer'].values[0])/ path.Path(picture)

    new_path = PATH_TARGET_IMAGES_FOLDER.parent / path.Path('unselected/'+'layer_' + id_bad['layer'].values[0])/ path.Path(picture)
    os.rename(old_path, new_path)


In [51]:
total_img_pool = contrast_similarity_updated[:,:,1].shape[0]
for layer in ['0','1','2']:

    current_target_img = target_image_layer_df_updated[target_image_layer_df_updated['layer'] == layer]
    total_images = current_target_img.shape[0]

    # if there are still 20 target images (no bad images from this layer), we continue to the next layer
    if total_images == 20:
        print(f'no missing values for layer {layer}, skipping to next layer')
        continue
    
    # for the layers that have less than 20 target images, we loop until we have filled the gap :
    while total_images < 20:
        selected_target_imgs = current_target_img['mapped_value'].values
        
        # looping over each image - in rows - looking only at the previosly selected images columns 
        # appending the mean to the list 
        total_mean_similarity = []
        for img in range(total_img_pool):
            mean_similarity_contrast = contrast_similarity_updated[img,selected_target_imgs,int(layer)].mean() 
            total_mean_similarity.append(mean_similarity_contrast)
        
        # finding the max mean value and index 
        max_similarity_val = max(total_mean_similarity)
        max_index = total_mean_similarity.index(max_similarity_val) 
        new_img_name = list(mapping_dict.keys())[list(mapping_dict.values()).index(max_index)]
        print(f'maximum value for layer {layer} is {max_similarity_val}')
        print(f'the index of the picture is {max_index}')
        print(f'the name of the image is {new_img_name}')

        # now that we have a new selected image , we put -9999 in that row so it isnt going to be selected again 
        contrast_similarity_updated[max_index,:,:] = -9999

        # update the df : add the new row to the df 
        target_image_layer_df_updated.loc[new_img_name] = [layer,max_index]
        current_target_img = target_image_layer_df_updated[target_image_layer_df_updated['layer'] == layer]
        total_images = current_target_img.shape[0]

# sorting values so layers will be together 
target_image_layer_df_updated = target_image_layer_df_updated.sort_values('layer')

no missing values for layer 0, skipping to next layer
maximum value for layer 1 is 2.424939283771811
the index of the picture is 408
the name of the image is 2446759347-347016.jpg
maximum value for layer 1 is 2.3344198338599083
the index of the picture is 49
the name of the image is 2449441881-272368.jpg
maximum value for layer 2 is 3.377413640240492
the index of the picture is 2270
the name of the image is 2883254439-140941.jpg
maximum value for layer 2 is 3.2792082526064186
the index of the picture is 549
the name of the image is 2901948350-218459.jpg


In [62]:
# saving new target images to folders 
# this part is done manualy because we dont have the whole dataset on the computer. we look through it online and download only the 
# selected pictures 

## try this.. dosent work so well. best to chane by hand , just wait 10 min for the image folder to load 
# moving image to new folder - unselected 
pictures = ['2446759347-347016.jpg',' 2449441881-272368.jpg','2883254439-140941.jpg','2901948350-218459.jpg']
layers = [1,1,2,2]

import shutil, os
os.mkdir('D:/my_new_folder')

for f in pictures:
    shutil.copy(f, 'D:/my_new_folder')


FileNotFoundError: [Errno 2] No such file or directory: '2446759347-347016.jpg'